<a href="https://colab.research.google.com/github/rizkynalfian/Sentiment_Analysis_Text_Mining/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import re
import string
import seaborn as sns
sns.set()
import warnings 
warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [ ]:
df_datatweet=pd.read_csv('https://raw.githubusercontent.com/rizkynalfian/Sentiment_Analysis_Text_Mining/master/IDHSD_RIO_unbalanced_713_2017.txt', sep='\t', encoding = "ISO-8859-1", dtype=str)
df_datatweet.head()

In [ ]:
df_datatweet.info()

In [ ]:
# Set Training and Testing Data (70:30)
X_train, X_test, y_train, y_test  = train_test_split(df_datatweet['Tweet'] , df_datatweet['Label'], shuffle = True, test_size=0.25, random_state=1)

# Show the Training and Testing Data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#Exploratory Data Analysis
df_datatweet['length'] = df_datatweet['Tweet'].apply(len)
fig1 = sns.barplot('Label','length',data = df_datatweet,palette='PRGn')
plt.title('Average Word Length vs label')
plot = fig1.get_figure()
plot.savefig('Barplot.png')
plt.show()

# **Memproses Data**

In [ ]:
! pip install Sastrawi

In [ ]:
df_datatweet_set_NonHS = df_datatweet.loc[df_datatweet['Label'] == 'Non_HS']
df_datatweet_set_NonHS = df_datatweet_set_NonHS.head(453)
df_datatweet_set_HS = df_datatweet.loc[df_datatweet['Label'] == 'HS']
df_datatweet_set_HS = df_datatweet_set_HS.head(259)
df_datatweetnew = df_datatweet_set_NonHS.append(df_datatweet_set_HS, ignore_index=True)
df_datatweetnew

In [ ]:
#grafik bar untuk menghitung label negatif positif
fig2 = sns.countplot(x= 'Label',data = df_datatweetnew)
plt.title('Label Counts')
plot = fig2.get_figure()
plt.show()

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import re
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
train=df_datatweet
train['text_lem'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]',' ',text)) for text in lis]) for lis in train['Tweet']]

In [ ]:
train['text_lem']

In [ ]:
from wordcloud import WordCloud,STOPWORDS
word = df_datatweet.Tweet[df_datatweet['Label']=='Non_HS'].astype('string')
wordcloud = WordCloud(max_font_size=30, max_words=500, background_color="white").generate_from_text(' '.join(word))
print('Non Hate speech')
plt.figure(figsize= (12,8))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
from wordcloud import WordCloud,STOPWORDS
word = df_datatweet.Tweet[df_datatweet['Label']=='HS'].astype('string')
wordcloud = WordCloud(max_font_size=30, max_words=500, background_color="white").generate_from_text(' '.join(word))
print('Hate speech')
plt.figure(figsize= (12,8))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

# **Cleaning the tweets**

In [ ]:
# Cleaning the tweets
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
def preprocessing(a,b):
    data_tweetnew= []
    from nltk.corpus import stopwords
    for i in range(len(b)):
        teks = re.sub('[^a-zA-Z]', ' ', a[i])
        # Remove mentions
        teks = re.sub(r'@[A-Za-z0-9_]+', '', a[i])
        # Remove hashtags
        teks = re.sub(r'#', '', a[i])
        # Remove retweets:
        teks = re.sub(r'RT : ', '', a[i])
        # Remove urls
        teks = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', a[i])
        #Case Folding
        teks = teks.lower()
        #Tokinize
        teks = teks.split()
        #Stopword + Stemming
        stemmer = StemmerFactory().create_stemmer()        
        teks = [stemmer.stem(word) for word in teks if not word in set(stopwords.words('indonesian'))]
        teks = [word for word in teks if len (word) > 3]
        teks = ' '.join(teks)
        data_tweetnew.append(teks)
    return data_tweetnew

In [ ]:
length_of_set = df_datatweetnew['Label'].tolist()

In [ ]:
data_set = preprocessing(df_datatweetnew['Tweet'],length_of_set)

In [ ]:
df_datatweetnew['Prepro'] = pd.Series(data_set).values
df_datatweetnew

# **Model Analisis Sentimen**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn import metrics
mnb = MultinomialNB()

In [ ]:
train=df_datatweetnew['Prepro']
train_data = TfidfVectorizer.fit_transform(train)
train_data.shape